### 두개의 투자 전략을 하나의 class 선언

1. 클래스의 이름은 Invest 상속은 X
2. 생성자 함수 
    - 매개변수 4개
        - 주식 데이터 (_df)
        - 기준이 되는 컬럼의 이름 (_col = 'Adj Close')
        - 투자의 시작 시간 (_start = '2010-01-01')
        - 투자의 종료 시간 (_end = datetime.now())
    - 결측치, 양의 무한대, 음의 무한대를 제외
    - 컬럼중 Date 컬럼이 존재한다면 Date 컬럼을 인덱스로 변환
    - index를 시계열 데이터로 변환
    - index에 tz에 존재한다면 tz를 None 변경
    - 시작 시간과 종료 시간은 시계열 데이터로 변환
        - 종료시간은 문자열인 경우에만 변환환
    - 기준이 컬럼(self.col), 시작시간(self.start), 종료시간(self.end)은 self변수로 생성
3. buyandhold 함수를 생성 
    - 매개변수 x (self를 생성)
    - 시작시간과 종료시간으로 인덱스 필터 기준이되는 컬럼을 제외하고 모두 제거 
    - 일별 수익율 컬럼을 생성하여 pct_change() + 1 함수를 이용하여 값을 대입
    - 누적수익율 컬럼을 생성하여 누적 수익율 데이터를 대입 
    - 만들어진 데이터프레임과 총 누적수익율을 되돌려준다.
4. bollinger 함수를 생성 
    - 매개변수 2개 
        - 신뢰구간 (_num = 2)
        - 데이터 개수 (_cnt = 20)
    - 특정컬럼(self.col) 컬럼을 제외한 나머지 컬럼을 제거하고 변수에 저장
    - 이동평균선, 상단밴드, 하단밴드 생성
    - 시작시간, 종료 시간을 기준으로 데이터 필터링 
    - 보유 내역 추가 
    - 데이터 프레임을 되돌려준다. 
5. 수익율을 계산하는 함수 생성 create_rtn()
    - 매개변수 1개 : bollinger()함수의 결과 데이터프레임
    - 복사본 생성 
    - rtn 컬럼을 생성해서 1을 대입 
    - 매수, 매도 시기를 확인하여 수익율 대입 
    - rtn을 기준으로 누적 수익율을 acc_rtn 컬럼에 대입 
    - 만들어진 데이터프레임과 최종수익율을 되돌려준다.


In [1]:
import pandas as pd 
from datetime import datetime
import numpy as np

In [2]:
import momentum as mnt

ModuleNotFoundError: No module named 'momentum'

In [42]:
import warnings
warnings.filterwarnings("ignore")

In [43]:
class Invest:
    # 생성자 함수 
    def __init__(
            self,
            _df, 
            _col = 'Adj Close', 
            _start = '2010-01-01', 
            _end = datetime.now()
    ):
        # 조건식 생성 
        flag = _df.isin( [np.nan, np.inf, -np.inf ] ).any(axis=1)
        self.df = _df.loc[~flag]
        # self.df의 컬럼중 Date가 존재한다면?
        if 'Date' in self.df.columns:
            self.df.set_index('Date', inplace=True)
        # index를 시계열 데이터로 변경 
        # index를 시계열로 변경할때 DatetimeIndex -> tz 가 존재 
        # index를 시계열로 변경할때 Index -> tz 존재 X
        self.df.index = pd.to_datetime(self.df.index, utc=True)
        # tz가 존재하는가?
        if self.df.index.tz:
            self.df.index = self.df.index.tz_localize(None)
        # 인자값으로 들어온 기준이 되는 컬럼의 이름은 self변수에 저장
        self.col = _col
        # 시작 시간은 시계열로 변경 
        try:
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end
        except:
            print("시간의 타입이 맞지 않습니다. 포멧은 YYYY-mm-dd 형식입니다.")
            print('Class를 다시 생성하시기 바랍니다.')
        
    def buyandhold(self):
        # 시작 시간, 종료시간 , 특정 컬럼을 기준으로 데이터를 필터 
        result = self.df.loc[self.start : self.end , [self.col]]
        # 일별 수익율 생성
        result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
        # 누적 수익율 생성 
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적 수익율을 변수에 저장 
        acc_rtn  = result.iloc[-1, -1]
        return result, acc_rtn
    
    def bollinger(
            self, 
            _num = 2, 
            _cnt = 20
    ):
        # 특정 컬럼을 기준으로 데이터를 필터
        result = self.df[[self.col]]
        # 이동 평균선 생성
        result['center'] = result[self.col].rolling(_cnt).mean()
        # 표준편차 값 변수에 저장 
        std_series = result[self.col].rolling(_cnt).std()
        # 상단밴드 생성
        result['up'] = result['center'] + (_num * std_series)
        # 하단밴드 생성
        result['down'] = result['center'] - (_num * std_series)
        # 시작 시간과 종료시간을 기준으로 데이터 필터
        result = result.loc[self.start : self.end]
        # 보유 내역 추가 
        result['trade'] = ''
        for idx in result.index:
            # 상단 밴드의 가격보다 기준이 되는 컬럼의 값이 크거나 같다면
            if result.loc[idx, self.col] >= result.loc[idx, 'up']:
                result.loc[idx, 'trade'] = ''
            # 하단밴드의 가격보다 기준이 되는 컬럼의 값이 작거나 같다면
            elif result.loc[idx, 'down'] >= result.loc[idx, self.col]:
                result.loc[idx, 'trade'] = 'buy'
            # 밴드 중간에 기준이되는 컬럼의 값이 존재한다면?
            else:
                # 현재 보유중이면 유지
                if result.shift().loc[idx, 'trade'] == 'buy':
                    result.loc[idx, 'trade'] = 'buy'
                # 보유중이 아니면 유지
                else:
                    result.loc[idx, 'trade'] = ''
        # class 내부의 create_rtn 함수를 호출 
        rtn_result, acc_rtn = self.create_rtn(result)
        return rtn_result, acc_rtn
    
    # 수익율을 계산하는 함수 
    def create_rtn(self, _df):
        # 복사본 생성 
        result = _df.copy()

        # rtn 컬럼을 생성 1을 대입 
        result['rtn'] = 1

        for idx in result.index:
            # 매수 
            if (result.shift().loc[idx, 'trade'] == '') &\
                (result.loc[idx, 'trade'] == 'buy'):
                buy = result.loc[idx, self.col]
                print(f"매수일 : {idx}, 매수가 : {buy}")
            # 매도
            elif (result.shift().loc[idx, 'trade'] == 'buy') &\
                (result.loc[idx, 'trade'] == ''):
                sell = result.loc[idx, self.col]
                print(f"매도일 : {idx}, 매도가 : {sell}")
                # 수익율 계산
                rtn = sell / buy
                # 수익율을 데이터프레임에 대입 
                result.loc[idx, 'rtn'] = rtn
                print(f"수익율 : {rtn}")
        # 누적수익율 계산하고 대입 
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적수익율을 변수에 저장 
        acc_rtn = result.iloc[-1, -1]
        return result, acc_rtn
    

    def momentum(
        self,
        _score = 1,
        _momentum = 12,
        _select = 1
    ):
        ym_df = mnt.create_ym(self.df, _col = self.col)
        month_df = mnt.create_month(ym_df,
                                    _start = self.start,
                                    _end = self.end,
                                    _select = _select,
                                    _momentum = _momentum
                                    )
        trade_df = mnt.create_trade(ym_df, month_df, _score = _score)
        result , acc_rtn = self.create_rtn(trade_df)
        return result, acc_rtn



In [44]:
df = pd.read_csv('../csv/csv/AAPL.csv')

In [45]:
test_invest = Invest(df)

In [46]:
test_invest.buyandhold()

(             Adj Close       rtn   acc_rtn
 Date                                      
 2010-01-04   26.782711  1.000000  1.000000
 2010-01-05   26.829010  1.001729  1.001729
 2010-01-06   26.402260  0.984094  0.985795
 2010-01-07   26.353460  0.998152  0.983973
 2010-01-08   26.528664  1.006648  0.990515
 ...                ...       ...       ...
 2019-06-18  198.449997  1.023518  7.409631
 2019-06-19  197.869995  0.997077  7.387975
 2019-06-20  199.460007  1.008036  7.447342
 2019-06-21  198.779999  0.996591  7.421952
 2019-06-24  199.169998  1.001962  7.436514
 
 [2384 rows x 3 columns],
 np.float64(7.436513727083075))

In [47]:
test_invest.bollinger()

매수일 : 2010-01-22 00:00:00, 매수가 : 24.747818
매도일 : 2010-03-01 00:00:00, 매도가 : 26.154476
수익율 : 1.0568396777445188
매수일 : 2010-08-24 00:00:00, 매수가 : 30.026524
매도일 : 2010-09-08 00:00:00, 매도가 : 32.90366
수익율 : 1.0958198158401553
매수일 : 2011-03-16 00:00:00, 매수가 : 41.299767
매도일 : 2011-07-01 00:00:00, 매도가 : 42.957966
수익율 : 1.040150323366231
매수일 : 2011-11-14 00:00:00, 매수가 : 47.463268
매도일 : 2011-12-27 00:00:00, 매도가 : 50.876015
수익율 : 1.0719029081604747
매수일 : 2012-04-16 00:00:00, 매수가 : 72.601524
매도일 : 2012-06-18 00:00:00, 매도가 : 73.308609
수익율 : 1.0097392583659814
매수일 : 2012-10-08 00:00:00, 매수가 : 80.207954
매도일 : 2013-07-29 00:00:00, 매도가 : 57.243137
수익율 : 0.7136840443530077
매수일 : 2013-09-11 00:00:00, 매수가 : 60.184383
매도일 : 2013-10-18 00:00:00, 매도가 : 65.48336
수익율 : 1.0880457144505413
매수일 : 2014-01-03 00:00:00, 매수가 : 70.019096
매도일 : 2014-03-25 00:00:00, 매도가 : 70.960335
수익율 : 1.0134426042861222
매수일 : 2014-10-15 00:00:00, 매수가 : 89.842468
매도일 : 2014-10-23 00:00:00, 매도가 : 96.557182
수익율 : 1.0747387527243797
매수일 

(             Adj Close      center          up        down trade  rtn  \
 Date                                                                    
 2010-01-04   26.782711   25.037723   27.046734   23.028713        1.0   
 2010-01-05   26.829010   25.169503   27.288098   23.050908        1.0   
 2010-01-06   26.402260   25.307290   27.366449   23.248130        1.0   
 2010-01-07   26.353460   25.436879   27.410937   23.462821        1.0   
 2010-01-08   26.528664   25.525609   27.529742   23.521475        1.0   
 ...                ...         ...         ...         ...   ...  ...   
 2019-06-18  198.449997  185.432500  201.032574  169.832427   buy  1.0   
 2019-06-19  197.869995  185.996000  202.558154  169.433846   buy  1.0   
 2019-06-20  199.460007  186.830000  204.361771  169.298229   buy  1.0   
 2019-06-21  198.779999  187.786000  205.751400  169.820600   buy  1.0   
 2019-06-24  199.169998  188.796000  206.944862  170.647138   buy  1.0   
 
              acc_rtn  
 Date       

In [48]:
# test_invest.create_rtn(boll_df)

In [49]:
test_invest.momentum()

매수일 : 2010-01-29 00:00:00, 매수가 : 24.035734
매도일 : 2013-02-28 00:00:00, 매도가 : 56.055256
수익율 : 2.3321632699047177
매수일 : 2013-12-31 00:00:00, 매수가 : 72.612869
매도일 : 2016-01-29 00:00:00, 매도가 : 91.581261
수익율 : 1.2612263123772178
매수일 : 2016-12-30 00:00:00, 매수가 : 111.392426
매도일 : 2019-01-31 00:00:00, 매도가 : 165.093445
수익율 : 1.4820886026847104
매수일 : 2019-03-29 00:00:00, 매수가 : 189.221313
매도일 : 2019-06-24 00:00:00, 매도가 : 199.169998
수익율 : 1.052576979000246


(             Adj Close   STD_YM trade       rtn   acc_rtn
 Date                                                     
 1980-12-12    0.410525  1980-12        1.000000  1.000000
 1980-12-15    0.389106  1980-12        1.000000  1.000000
 1980-12-16    0.360548  1980-12        1.000000  1.000000
 1980-12-17    0.369472  1980-12        1.000000  1.000000
 1980-12-18    0.380182  1980-12        1.000000  1.000000
 ...                ...      ...   ...       ...       ...
 2019-06-18  198.449997  2019-06   buy  1.000000  4.359394
 2019-06-19  197.869995  2019-06   buy  1.000000  4.359394
 2019-06-20  199.460007  2019-06   buy  1.000000  4.359394
 2019-06-21  198.779999  2019-06   buy  1.000000  4.359394
 2019-06-24  199.169998  2019-06        1.052577  4.588598
 
 [9713 rows x 5 columns],
 np.float64(4.588597970528201))